In [20]:
import pandas as pd 


In [21]:
PREFIX = '11/artem-pyanykh-cmc-prac-task3-seed17/out/input/'
SAVE_PREFIX = './answer/'
list_of_start_names_of_files = ['MS-b1','MS-b2','MS-m1',
                                'MS-m2','MS-s1','MS-s2',
                                'MS-s3','MS-s4','MS-s5'];

In [28]:
def sell_parse(sell):
    sell['sku_num'] = sell['sku_num'].apply(lambda x: -1 if x.find('ap') == 6 else 1)
    sell['num'] = sell['sku_num']
    sell['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x > 0 else -x)
    sell['num'] = sell['num'].apply(lambda x: 0 if x < 0 else x)
    sell = sell.rename(columns={"sku_num": "apple", "num": "pen"})
    sell = sell.groupby(['date']).sum()
    return sell

In [29]:
def daily_inventary(supply, sell):
    daily_inv = pd.DataFrame(columns=['apple', 'pen'], index = sell.index)
    daily_inv['apple'] = -sell['apple']
    daily_inv['pen'] = -sell['pen']
    daily_inv = pd.concat([daily_inv, supply])
    daily_inv = daily_inv.groupby(['date']).sum()
    daily_inv['apple'] = daily_inv['apple'].cumsum()
    daily_inv['pen'] = daily_inv['pen'].cumsum()
    return daily_inv

In [30]:
def month_steal(daily_inv, inventory):
    month_inv = daily_inv.resample('M').last()
    month_inv['pen'] -= inventory['pen']
    month_inv['apple'] -= inventory['apple']
    month_inv['pen'] = month_inv['pen'] - month_inv['pen'].shift(1).fillna(0)
    month_inv['apple'] = month_inv['apple'] - month_inv['apple'].shift(1).fillna(0)
    return month_inv

In [31]:
def update_global_statistic_frame(state, sell, steal, global_statistic_frame):
    sell1 = sell.resample('Y').sum()
    sell1 = sell1.rename(columns={"apple": "apple_sold", "pen": "pen_sold"})
    sell1['state'] = ''
    sell1['year'] = sell1.index
    sell1['year'] = sell1['year'].apply(lambda x: x.to_pydatetime().year)
    sell1.index = range(0, len(sell1.index))
    
    steal1 = steal.resample('Y').sum()
    steal1['state'] = state
    steal1 = steal1.rename(columns={"apple": "apple_stolen", "pen": "pen_stolen"})
    steal1['year'] = steal1.index
    steal1['year'] = steal1['year'].apply(lambda x: x.to_pydatetime().year)
    steal1.index = range(0, len(steal1.index))
    
    m_year_res = steal1.append(sell1, ignore_index=True, sort=False).groupby(['year']).sum(skipna=True)
    m_year_res['year'] = m_year_res.index
    m_year_res.index = range(0, len(m_year_res.index))
    
    global_statistic_frame = global_statistic_frame.append(m_year_res, ignore_index=True,
                sort=False).groupby(['year', 'state'], as_index=False)['apple_sold', 
                                                                       'apple_stolen', 
                                                                       'pen_sold', 
                                                                       'pen_stolen'].sum(skipna=True)
    return global_statistic_frame

In [32]:
#get tables and start_process
def start(name, global_statistic_frame):
    j=PREFIX+name;
    supply = pd.read_csv(j+'-supply.csv', sep =',')
    supply.index = supply.date
    supply = supply.drop('date', axis=1)
    sell = pd.read_csv(j+'-sell.csv', sep =',')
    inventory = pd.read_csv(j+'-inventory.csv', sep =',')
    inventory.index = inventory.date
    inventory = inventory.drop('date', axis=1)
    sell = sell_parse(sell)
    sell.index = pd.to_datetime(sell.index)
    inventory.index = pd.to_datetime(inventory.index)
    supply.index = pd.to_datetime(supply.index)
    
    #print(name + '[sell]:\n' , sell.head(), "\n")
    #print(name + '[supply]:\n' ,  supply.head(), "\n")
    #print(name + '[inventory]:\n' , inventory.head(), "\n")
    
    di = daily_inventary(supply, sell)
    di.to_csv(SAVE_PREFIX+name+'-daily.csv', index=True)
    print(name + '[daily_inventary]:\n' , di.head(), "\n")
    ms = month_steal(di, inventory)
    ms.to_csv(SAVE_PREFIX+name+'-steal.csv', index=True)
    print(name + '[month_steal]:\n' , ms.head(), "\n")
    return update_global_statistic_frame(name[0:2], sell, 
                                         ms ,global_statistic_frame)

In [33]:
global_statistic_frame = pd.DataFrame(columns=['year', 'state',
                                               'apple_sold','apple_stolen',
                                               'pen_sold', 'pen_stolen'])
for i in list_of_start_names_of_files:
    global_statistic_frame = start(i, global_statistic_frame)
    print('\n----------------------------------------\n')
global_statistic_frame.to_csv(SAVE_PREFIX+'global.csv', index=True)
print(global_statistic_frame)

MS-b1[daily_inventary]:
             apple   pen
date                   
2006-01-01  33271  2574
2006-01-02  31409  2431
2006-01-03  29529  2260
2006-01-04  27732  2107
2006-01-05  25790  1974 

MS-b1[month_steal]:
             apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0 


----------------------------------------

MS-b2[daily_inventary]:
             apple   pen
date                   
2006-01-01  33271  2574
2006-01-02  31409  2431
2006-01-03  29529  2260
2006-01-04  27732  2107
2006-01-05  25790  1974 

MS-b2[month_steal]:
             apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0 


----------------------------------------

MS-m1[daily_inventary]:
             apple  pen
date                  
2006-01-01  14244  767
2006-01-02  13395  722
2006-01-03  12475  689
20